In [ ]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import allantools as at
from typing import Callable, Iterable, Optional, Tuple
from functools import partial
from runData import runData

In [ ]:
pd.set_option("display.precision", 15)      # Show up to 15 decimal places
#baseName = 'fixedL1l53'
#baseName = 'baseline1'
dirName  = 'labData/'
#rowLimit = 2000 # Rows to keep after joining ticc and timTp samples

In [ ]:
# Big Picture
#   Read TICC data from runs of interest
#   Process it, do simple analysis without timTp data
#   Read timTp data
#   Join with TICC data and analyze
#   Plot results

In [ ]:
# Read TAPR TICC data into a dataframe, as captured to a file by ticc.py running on a host.
# Events are rising edges of a PPS signal from a DUT, resulting in a timestamp on the TICC's reference clock.
# Columns:
#  ppsHostClock: Host clock when serial data for PPS event timestamp was read (ticc.py logs this in UTC)
#  ppsRefClock:  Reference clock when PPS event happened (elapsed seconds since TICC started)
#
# The frequency of the TICC reference clock comes from an external 10 MHz source, a Geppetto Electronics GNSSDO in my case.
# It should be almost exactly 1e7 times the PPS frequency, with an arbitray phase relationship.
# So we expect the whole number portion of ppsRefClock to increment by 1 every second, while the fractional seconds jitter around
# some slowly-changing phase offset.
# Therefore, there's very little information in the whole seconds, while the fractional seconds contain the most interesting data.
# And as the whole number grows with a floating point representation, precision is lost in the fractional digits.
# So once we confirm the whole number of seconds is behaving as expected, we can drop it and focus on the fractional seconds.
# Whatever slowly-changing phase offset exists, it won't impact the deviation metrics.
# We treat the fractional seconds as an instantaneous (but nosiy) measurement of the phase error against the ref clock.
#
# There is a very small, but non-zero chance that the static phase offset plus the jitter causes sequential PPS timestamps to be
# within the same second or more than one second apart, leading to missing or duplicate whole seconds.
# Instead of properly handling whole seconds when this happens, just fail on assertions.
def readTicc(baseName, chan):
    ticcFile = f"{dirName}/{baseName}.ticc{chan}.csv"
    ticcData = pd.read_csv(ticcFile, dtype={'ppsHostClock': str, 'ppsRefClock': str})

    # Convert host timestamp string to UTC timestamp
    ticcData["ppsHostClock"] = pd.to_datetime(ticcData.ppsHostClock, utc=True)

    # Assuming host clock sync is better than serialization latency of timestamp arriving,
    # floor of host clock second will be the navigation epoch sencond.
    # Will be used for later join with TIM-TP timestamps.
    ticcData["epochSec"] = ticcData["ppsHostClock"].dt.floor("s")


    # Split ppsRefClock string into whole and fractional seconds
    ticcData[["rcWhole", "rcFrac"]] = ticcData.ppsRefClock.str.split(".", n=1, expand=True)

    # Check for missing or duplicate whole seconds
    ticcData['rcWhole'] = ticcData['rcWhole'].astype(int)
    expected = set(range(ticcData.rcWhole.min(), ticcData.rcWhole.max() + 1))
    observed = set(ticcData.rcWhole)
    missing = sorted(expected - observed)
    duplicates = ticcData.rcWhole[ticcData.rcWhole.duplicated()].unique().tolist()
    assert len(missing)    == 0, f"Missing whole seconds in ticc{chan}: {missing}"
    assert len(duplicates) == 0, f"Duplicate whole seconds in ticc{chan}: {duplicates}"

    # Convert ref clock fractional part from digit string to float
    ticcData['rcFrac'] = "0." + ticcData['rcFrac'].astype(str)
    ticcData['rcFrac'] = ticcData['rcFrac'].astype(float)

    # Also get fractional part of host clock
    ticcData['hcFrac'] = (ticcData.ppsHostClock.astype('int64')-1e9*(ticcData.ppsHostClock.astype('int64')//1e9))/1e9

    # With overly careful consideration of maintining floating point precision, get interval between PPS events on ref clock.
    ticcData['rcTi'] = (ticcData.rcWhole-ticcData.rcWhole.shift(1)) + (ticcData.rcFrac - ticcData.rcFrac.shift(1)) # Time interval between refClock samples on ref clock

    ticcData['bn'  ] = baseName
    ticcData['dut' ] = runData[baseName][chan]
    ticcData['chan'] = chan
    return ticcData

In [ ]:
dfs = []
dfs.append(readTicc('fixedL1l52', 'A'))
dfs.append(readTicc('fixedL1l52', 'B'))
dfs.append(readTicc('fixedL1l53', 'A'))
dfs.append(readTicc('fixedL1l53', 'B'))
dfs.append(readTicc('fixedPos1', 'A'))
dfs.append(readTicc('fixedPos2', 'A'))
ticc = pd.concat(dfs, ignore_index=True)

In [ ]:
ticc
# XXX next step: define function to generate statistics from selected rows of ticc, plot, and label them

In [ ]:
# Bring data from TICC channels for each run together for each ref clock second.
ticcA = ticc[ticc.chan == 'A'][['bn', 'rcWhole', 'rcFrac', 'epochSec']]
ticcB = ticc[ticc.chan == 'B'][['bn', 'rcWhole', 'rcFrac'            ]]

# Perform inner join on bn and rcWhole
rcSec = pd.merge(ticcA, ticcB, on=['bn', 'rcWhole'], how='outer', suffixes=('A', 'B'))

# Derive columns of interest from the TICC data.
if ticcB.empty:
    rcSec.drop(columns=['rcFracB'], inplace=True)
else:
    rcSec['rcFracAB'] = rcSec.rcFracA - rcSec.rcFracB  # Phase difference between the two channels

In [ ]:
ticcB

In [ ]:
rcSec

In [ ]:
# XXX Fixme when run with no channel B
beg = 1000
end = 2000
bn = 'fixedL1l53'
plt.figure(figsize=(24, 6))
plt.plot(rcSec[rcSec.bn==bn].epochSec[beg:end], rcSec[rcSec.bn==bn].rcFracAB[beg:end]*1e9, marker='.', linestyle='-', color='b')
plt.title(f"Phase Difference (rcFracAB) vs Epoch Second for Run {bn}")
plt.xlabel('Epoch Second')
plt.ylabel('Phase Difference (rcFracAB)')
plt.grid()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(22, 6))
fig.canvas.draw()
bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
widthPx = int(bbox.width*fig.dpi)
print("Figure width in pixels:", widthPx)
print(fig.get_figwidth())
for bn in ticc.bn.unique():
    ticcSubset = ticc[ticc.bn == bn]
    plt.hist(1e9*(ticcSubset.rcTi-1.0), bins=widthPx, label=bn, alpha=0.5)
#plt.hist(1e9*(ticc[ticc.bn==bn].rcTi[beg:end]-1.0), bins=widthPx, color='blue', alpha=0.7)
plt.title('Histogram of Time Error Between PPS Pulses')
plt.xlabel('Time Error (ns)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.legend()
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:g} ns"))
plt.show()



In [ ]:
def readTimTp(baseName, chan):
    timTp = pd.read_csv(f"{dirName}/{baseName}.timTp{chan}.csv")

    # Confirm expected values in constant columns, then drop them
    assert (timTp['timeBase'   ] ==  1).all(), "Not all rows in timTp.timeBase are equal to 1"
    assert (timTp['utc'        ] ==  1).all(), "Not all rows in timTp.utc are equal to 1"
    assert (timTp['raim'       ] ==  2).all(), "Not all rows in timTp.raim are equal to 2"
    assert (timTp['qErrInvalid'] ==  0).all(), "Not all rows in timTp.qErrInvalid are equal to 0"
    assert (timTp['TpNotLocked'] ==  0).all(), "Not all rows in timTp.TpNotLocked are equal to 0"
    assert (timTp['timeRefGnss'] == 15).all(), "Not all rows in timTp.timeRefGnss are equal to 15"
    assert (timTp['utcStandard'] ==  3).all(), "Not all rows in timTp.utcStandard are equal to  3"
    assert (timTp['towSubMS'   ] ==  0).all(), "Not all rows in timTp.towSubMS are equal to  0"
    timTp.drop(columns=['timeBase', 'utc', 'raim', 'qErrInvalid', 'TpNotLocked', 'timeRefGnss', 'utcStandard', 'towSubMS'], inplace=True)

    # Constants for time conversion
    gps_epoch = pd.Timestamp("1980-01-06 00:00:00", tz="UTC")
    leap_seconds = pd.Timedelta(seconds=18)  # current GPS-UTC offset (2025)

    # Vectorized conversion from GPS week and TOW to epoch seconds
    timTp["epochSec"] = (
        gps_epoch
        + pd.to_timedelta(timTp.week  * 7, unit="D" )
        + pd.to_timedelta(timTp.towMS    , unit="ms")
    )
    timTp.drop(columns=['week', 'towMS'], inplace=True)

    timTp['qErrFrac'] = timTp.qErr/1e12 # Convert qErr from picoseconds to seconds

    timTp['bn'] = baseName
    timTp['dut' ] = runData[baseName][chan]
    timTp['chan'] = chan
    return timTp

In [ ]:
dfs = []
dfs.append(readTimTp('fixedL1l52', 'A'))
dfs.append(readTimTp('fixedL1l52', 'B'))
dfs.append(readTimTp('fixedL1l53', 'A'))
dfs.append(readTimTp('fixedL1l53', 'B'))
dfs.append(readTimTp('fixedPos1', 'A'))
dfs.append(readTimTp('fixedPos2', 'A'))
timTp = pd.concat(dfs, ignore_index=True)

In [ ]:
timTp

In [ ]:
# Bring data from TIM-TP messages for each run together for each epoch second.
timTpA = timTp[timTp.chan == 'A'][['bn', 'epochSec', 'qErr', 'qErrFrac']]
timTpB = timTp[timTp.chan == 'B'][['bn', 'epochSec', 'qErr', 'qErrFrac']]

# Perform inner join on bn and epoch second
epSec = pd.merge(timTpA, timTpB, on=['bn', 'epochSec'], how='outer', suffixes=('A', 'B'))
if timTpB.empty:
    epSec.drop(columns=['qErrB'    ], inplace=True)
    epSec.drop(columns=['qErrFracB'], inplace=True)

In [ ]:
# Merge TICC data from above with TIM-TP data on epoch second.
epSec = pd.merge(epSec, rcSec, on=['bn', 'epochSec'], how="inner")

In [ ]:
epSec

In [ ]:
# Correct rcFrac with qErr
epSec['rcFracCorrA'] = epSec.rcFracA+epSec.qErrFracA

if not timTpB.empty:
    epSec['rcFracCorrB'] = epSec.rcFracB+epSec.qErrFracB
    # Corrected phase difference between the two channels
    epSec['rcFracCorrAB'] = epSec.rcFracCorrA - epSec.rcFracCorrB

In [ ]:
epSec

In [ ]:
if not timTpB.empty:
    for bn in epSec.bn.unique():
        print(f"Stats for run: {bn}")
        print(epSec[epSec.bn==bn][:1000].rcFracCorrAB.describe()[1:]*1e9)

In [ ]:
# If I have two series of PPS timestamps from imperfect clocks and I want to compare how well they track each other, paying less attention to the absolute phase error and more to the relative phase error, are there statistical analysis methods like Allan deviation that I can apply to the difference between clocks?

In [ ]:
beg = 11000
end = 11500
bn = 'fixedL1l53'
plt.figure(figsize=(24, 6))
plt.plot(epSec[epSec.bn==bn].epochSec[beg:end], epSec[epSec.bn==bn].rcFracAB[beg:end]*1e9, marker='.', linestyle='-', color='b')
plt.plot(epSec[epSec.bn==bn].epochSec[beg:end], epSec[epSec.bn==bn].rcFracCorrAB[beg:end]*1e9, marker='.', linestyle='-', color='r')
plt.title(f"Phase Difference (epFracAB) vs Epoch Second for Run {bn}")
plt.xlabel('Epoch Second')
plt.ylabel('Phase Difference (epFracAB)')
plt.grid()
plt.show()


In [ ]:
def statPlotStart(
    title: Optional[str] = None,
    xlabel: str = r"$\tau$",
    ylabel: Optional[str] = r"$\sigma_x(\tau)$",
    logx: bool = True,
    logy: bool = True,
    grid: bool = True,
    figsize: Tuple[float, float] = (16, 9),
) -> Tuple[plt.Figure, plt.Axes]:

    sns.set(style="whitegrid")
    fig, ax = plt.subplots(figsize=figsize)
    if logx and logy:
        ax.set_xscale("log")
        ax.set_yscale("log")
    elif logx:
        ax.set_xscale("log")
    elif logy:
        ax.set_yscale("log")
    if title:
        ax.set_title(title)
    ax.set_xlabel(xlabel)
    if ylabel:
        ax.set_ylabel(ylabel)
    if grid:
        ax.grid(True, which="both", alpha=0.35)

    return fig, ax

In [ ]:
# Type: any callable that takes your prepared data and returns (taus, values, errors_or_None)
StatFn = Callable[[np.ndarray], Tuple[np.ndarray, np.ndarray, Optional[np.ndarray]]]

def statPlotTrace(
    ax: plt.Axes,
    s: pd.Series,
    stat_fn: StatFn,
    secLimit: Optional[int] = sys.maxsize,
    *,
    label: Optional[str] = None,
    # Matplotlib styling (pass whatever you like; e.g., color, linestyle, marker, alpha, linewidth ...)
    **line_kwargs,
) -> Tuple[np.ndarray, np.ndarray, Optional[np.ndarray]]:
    """
    Preprocesses the Series, calls stat_fn, then plots (taus, value) on ax.
    Returns (taus, values, errors_or_None) for further use if needed.
    """
    taus, vals, errs = stat_fn(s[:secLimit])  # Apply the statistic function to the Series, limited by secLimit

    # Plot the statistic (errs unused here; you can add errorbars if desired)
    # ax.plot(taus, 1e9*vals, label=label, **line_kwargs)
    plt.loglog(taus, 1e9*vals, label=label, **line_kwargs)
    return taus, vals, errs

In [ ]:
def statPlotFinish(
    ax: plt.Axes,
    legend: bool = True,
    legend_loc: str = "best",
    tight_layout: bool = False,
):
    if legend:
        ax.legend(loc=legend_loc)
    if tight_layout:
        ax.figure.tight_layout()

    #plt.annotate(f"Uncorrected TDEV(1 s): {tdevUcNs[0]:.3f} ns", xy=(1, tdevUcNs[0]), xytext=(1.1, 3),
    #            arrowprops=dict(arrowstyle="fancy", ec="black", fc="yellow", lw=0.5))
    #plt.annotate(f"Corrected TDEV(1 s): {tdevCorrNs[0]:.3f} ns", xy=(1, tdevCorrNs[0]), xytext=(1.1, 0.25),
    #            arrowprops=dict(arrowstyle="fancy", ec="black", fc="yellow", lw=0.5))

    plt.grid(which="major", linestyle="-" , linewidth=1.0, color="gray"     )
    plt.grid(which="minor", linestyle="--", linewidth=1.0, color="lightgray")

    plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:g} s"))
    plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:g} ns"))

    # Shade noise floor
    ymin, ymax = plt.ylim() # Save limits before shading
    yShadeMin = ymin
    yShadeMax = 0.1
    plt.axhspan(yShadeMin, yShadeMax, color="lightgray", alpha=0.3)

    # Add label centered inside shaded region, remembering that geometric mean is midpoint on log scales
    y_center = np.sqrt(yShadeMin * yShadeMax)  # geometric mean for log scale
    x_center = np.sqrt(plt.xlim()[0] * plt.xlim()[1])  # vertical center in log scale
    plt.text(
        x_center, y_center, "RMS Jitter Floor of TAPR TICC",
        ha="center", va="center",
        fontsize=10, color="black",
        bbox=dict(facecolor="white", alpha=0.7, edgecolor="none")
    )
    plt.ylim(ymin, ymax) # Restore limits after shading

    plt.text(
        0.95, 0.1,                # X & Y position in axes fraction (0–1)
        "Instrument: TAPR TICC\nReference: Geppetto GPSDO with OH300 5ppb OCXO",
        ha="right", va="bottom",   # Align to lower-right corner
        multialignment="left",
        transform=plt.gca().transAxes,  # ✅ Use axes fraction (not data coords)
        fontsize=10,
        bbox=dict(
            facecolor="white",     # Background color
            edgecolor="black",     # Border color
            boxstyle="round,pad=0.3"  # Rounded box with padding
        )
    )
    #    return ax.figure, ax

In [ ]:
# Adapters — Wrap allantools functions to match StatFn signature.
# Tip: use functools.partial to pre-set rate, data_type, taus, overlapping, etc.

def wrap_oadev(rate: float = 1.0, data_type: str = "phase", taus: Optional[Iterable[float]] = None, **kwargs) -> StatFn:
    """
    Returns a callable that takes 'data' and produces (taus, values, errors_or_None).
    """
    def _fn(data: np.ndarray) -> Tuple[np.ndarray, np.ndarray, Optional[np.ndarray]]:
        # Many versions return a tuple: (taus, stat, stat_err, ns)
        # If yours returns a dict, adjust the extraction below accordingly.
        out = at.oadev(data, rate=rate, data_type=data_type, taus=taus, **kwargs)
        try:
            tau, val, err, _ = out  # common tuple form
        except Exception:
            # Fallback for dict-like forms; update keys to match your version
            tau = np.asarray(out.get("taus") or out.get("tau"))
            val = np.asarray(out.get("oadev") or out.get("adev") or out.get("values"))
            err = out.get("oadev_err") or out.get("adeverror") or None
            if err is not None:
                err = np.asarray(err)
        return np.asarray(tau), np.asarray(val), (None if err is None else np.asarray(err))
    return _fn

def wrap_tdev(rate: float = 1.0, taus: Optional[Iterable[float]] = 'all', **kwargs) -> StatFn:
    def _fn(data: np.ndarray):
        out = at.tdev(data, rate=1.0, taus=taus, **kwargs)
        try:
            tau, val, err, _ = out
        except Exception:
            tau = np.asarray(out.get("taus") or out.get("tau"))
            val = np.asarray(out.get("tdev") or out.get("values"))
            err = out.get("tdev_err") or None
            if err is not None:
                err = np.asarray(err)
        return np.asarray(tau), np.asarray(val), (None if err is None else np.asarray(err))
    return _fn

In [ ]:
fig, ax = statPlotStart(title = r"Impact of u-blox F9T $\mathtt{qErr}$ Corrections on TDEV($\tau$)")

tdev = wrap_tdev(rate=1.0, taus='all')

statPlotTrace(ax, epSec[epSec.bn=='fixedPos1'].rcFracA    , tdev, secLimit=2000, label='Uncorrected', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedPos1'].rcFracCorrA, tdev, secLimit=2000, label='Corrected'  , linestyle='-', linewidth=2)
statPlotFinish(ax)

In [ ]:
fig, ax = statPlotStart(title = r"Comparing Two u-blox F9T $\mathtt{qErr}$ Corrections on TDEV($\tau$)")

tdev = wrap_tdev(rate=1.0, taus='all')

#statPlotTrace(ax, epSec[epSec.bn=='fixedL1l52'].rcFracA    , tdev, secLimit=8000, label='Run 2 Uncorrected Bob', linestyle='-', linewidth=2)
#statPlotTrace(ax, epSec[epSec.bn=='fixedL1l52'].rcFracB    , tdev, secLimit=8000, label='Run 2 Uncorrected PT', linestyle='-', linewidth=2)
#statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracA    , tdev, secLimit=8000, label='Run 3 Uncorrected Bob', linestyle='-', linewidth=2)
#statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracB    , tdev, secLimit=8000, label='Run 3 Uncorrected PT', linestyle='-', linewidth=2)

# Same
#statPlotTrace(ax, epSec[epSec.bn=='fixedL1l52'].rcFracCorrA, tdev, secLimit=3000, label='Run 2 Corrected Bob'  , linestyle='-', linewidth=2)
#statPlotTrace(ax, epSec[epSec.bn=='fixedL1l52'].rcFracCorrB, tdev, secLimit=3000, label='Run 2 Corrected PT'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrA, tdev, secLimit=3000, label='Run 3 Corrected Bob'  , linestyle='-', linewidth=2)
# Different
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrB, tdev, secLimit=3000, label='Run 3 Corrected PT'  , linestyle='-', linewidth=2)

statPlotTrace(ax, epSec[epSec.bn=='fixedL1l52'].rcFracCorrAB.dropna(), tdev, secLimit=3000, label='Run 2 Corrected AB'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrAB.dropna(), tdev, secLimit=3000, label='Run 3 Corrected AB'  , linestyle='-', linewidth=2)
statPlotFinish(ax)

In [ ]:
# XXX Next: clear output and checkpoint to git
# Try ADEV and MTIE
# Plot M600
# Collect data on CNS Clock
# Compare to baseline 5 without fixed position
# Collecte data on AliExpress OCXO
# Review other devices in talk proposal for data collection
# Study time error histograms more carefully
# Can you get error bars on TDEV and other plots?
# Get deeper understanding of different deviations

In [ ]:
epSec[epSec.bn=='fixedL1l52'].rcFracCorrAB.dropna()